In [1]:
#KNN Regression 

import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
import numpy as np
import plotly.io
from sklearn.utils import shuffle
#import data, X = inputs, Y = outputs 
df = pd.read_excel('ANN_TRAIN_TEST_RAMON_T40_AIR_WATER.xlsx')
df = shuffle(df)#embaralha os dados

X = df.iloc[:,1:7].values
Y = df.iloc[:, [12,13]].values

#normalize data
from sklearn.preprocessing import StandardScaler
scalerx = StandardScaler()
scalery = StandardScaler()
X_scaled = scalerx.fit_transform(X)
Y_scaled = scalery.fit_transform(Y)

In [10]:
# aCreating K-folds n applying knn regression

from sklearn.model_selection import KFold
kf = KFold(n_splits = 10)
from sklearn.metrics import r2_score, mean_squared_error
resultados = []
fold  = 0
for train, test in kf.split(X_scaled):
    fold+=1

    Xtrain = X_scaled[train]
    Ytrain = Y_scaled[train]

  
    #gridsearchcv aplicar no xtrain ytrain
    Ytest = Y_scaled[test]
    Xtest = X_scaled[test]

    
    from sklearn.neighbors import KNeighborsRegressor
    knn = KNeighborsRegressor(n_neighbors= 5, weights= 'uniform', metric='minkowski', p=2)
    knn.fit(Xtrain, Ytrain) #training the data

    Xtest_original = scalerx.inverse_transform(Xtest)
    Ytest_original = scalery.inverse_transform(Ytest)
    Xtrain_original = scalerx.inverse_transform(Xtrain)
    Ytrain_original = scalery.inverse_transform(Ytrain)

    Qpred = knn.predict(Xtest)
    Qpred_inverse = scalery.inverse_transform(Qpred)


    rmse = mean_squared_error(Ytest, Qpred, squared = False)
    r2 = r2_score(Ytest, Qpred)
    resultados.append((fold, rmse, r2, Xtest_original, Qpred_inverse))

    #plotando todas combinações
    trace_Xtrain = go.Scatter(x=Xtrain_original[:, 0], y=Ytrain_original[:, 0], mode='markers', name='Dados de Treino')
    trace_Xtest = go.Scatter(x=Xtest_original[:, 0], y=Qpred_inverse[:, 0], mode='markers', name='Dados de Teste')

# Criar layout
    layout = go.Layout(title='Gráfico Para Fold = {}'.format(fold),
                   xaxis={'title': 'Vazão mássica do ar'},
                   yaxis={'title': 'Taxa de Troca Térmica'})

    # Criar figura e plotar
    fig = go.Figure(data=[trace_Xtrain, trace_Xtest], layout=layout)
    pyo.plot(fig, filename='grafico_fold_{}.html'.format(fold))


In [8]:
print(Xtest[0])

[ 0.16992466 -0.04834865 -0.85979655 -0.48976395  0.65085424 -0.41579479]


In [8]:
for result in resultados:
    print(f"FOLD{result[0]} - RMSE = {result[1]}, R2 = {result[2]}")
    

FOLD1 - RMSE = 0.042286401729023884, R2 = 0.9981200586760246
FOLD2 - RMSE = 0.042172234995054625, R2 = 0.9982571551047154
FOLD3 - RMSE = 0.04416660206262056, R2 = 0.9981220385482474
FOLD4 - RMSE = 0.04542473440101388, R2 = 0.9978071709016623
FOLD5 - RMSE = 0.04863599626257263, R2 = 0.9976562318868738
FOLD6 - RMSE = 0.04352132766097408, R2 = 0.9982456086743318
FOLD7 - RMSE = 0.041625947322461165, R2 = 0.9982536832822052
FOLD8 - RMSE = 0.04247693163307617, R2 = 0.9981305954670442
FOLD9 - RMSE = 0.04839451423669744, R2 = 0.9975965493881767
FOLD10 - RMSE = 0.042721667645736656, R2 = 0.9980384674514952
